In [8]:
import streamlit as st
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
import pytz

# Optional scheduling
from apscheduler.schedulers.background import BackgroundScheduler


# ------------------------------------------------------------------------------
# 1) SCRAPE MAIN FIXTURE LIST (live vs. upcoming)
# ------------------------------------------------------------------------------
def get_match_data():
    """
    Scrapes the main fixture list page (https://crex.live/fixtures/match-list).
    Returns two lists:
      - live_data: Info about currently live matches
      - upcoming_data: Info about future matches
    """

    webdriver_path = "chromedriver.exe"
    service = Service(webdriver_path)
    options = Options()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(service=service, options=options)
    url = "https://crex.live/fixtures/match-list"
    driver.get(url)

    live_data = []
    upcoming_data = []

    try:
        # Wait until the match cards are present
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "match-card-container"))
        )

        soup = BeautifulSoup(driver.page_source, "html.parser")
        matches = soup.find_all(class_="match-card-container")

        for match in matches:
            # Check if match is LIVE
            if match.find(class_="liveTag"):
                link_tag = match.find("a", href=True)
                href = "https://crex.live" + link_tag["href"] if link_tag else ""

                teams_div = match.find_all("div", class_="team-info")
                team_name = []
                team_overs = []
                team_scores = []

                for t_div in teams_div:
                    name_el = t_div.find(class_="team-name")
                    over_el = t_div.find(class_="total-overs")
                    score_el = t_div.find(class_="team-score")

                    name = name_el.text.strip() if name_el else "N/A"
                    over = over_el.text.strip() if over_el else "Yet to bat"
                    score = score_el.text.strip() if score_el else "N/A"

                    team_name.append(name)
                    team_overs.append(over)
                    team_scores.append(score)

                live_data.append(
                    {
                        "live": True,
                        "name": team_name,
                        "over": team_overs,
                        "scores": team_scores,
                        "link": href,
                    }
                )

            # Check if match is UPCOMING
            elif match.find(class_="not-started"):
                link_tag = match.find("a", href=True)
                href = "https://crex.live" + link_tag["href"] if link_tag else ""

                time_start_el = match.find(class_="start-text")
                match_type_el = match.find(class_="time")

                time_start = time_start_el.text.strip() if time_start_el else "N/A"
                match_type = match_type_el.text.strip() if match_type_el else "N/A"

                teams_div = match.find_all("div", class_="team-info")
                team_name = [
                    (
                        t_div.find(class_="team-name").text.strip()
                        if t_div.find(class_="team-name")
                        else "N/A"
                    )
                    for t_div in teams_div
                ]

                upcoming_data.append(
                    {
                        "time_start": time_start,
                        "type": match_type,
                        "name": team_name,
                        "link": href,
                    }
                )

    finally:
        driver.quit()

    return live_data, upcoming_data


# ------------------------------------------------------------------------------
# 2) SCRAPE “MATCH INFO” TAB => /info
# ------------------------------------------------------------------------------
def scrape_match_info(info_url):
    """
    Scrapes data from the "Match Info" tab at (match_url + "/info").
    Typically includes toss, venue, series, date, etc.
    """

    webdriver_path = "chromedriver.exe"
    service = Service(webdriver_path)
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(service=service, options=options)
    driver.get(info_url)  # e.g., "https://crex.live/<match-id>/info"

    try:
        # Wait for the match info container
        try:
            WebDriverWait(driver, 30).until(  # Increase timeout to 30 seconds
                EC.presence_of_element_located((By.CLASS_NAME, "match-info-card"))
            )
        except TimeoutException:
            print("Element not found within the timeout period")
            return {} 
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 1) Venue & Date
        match_venue_el = soup.find(class_="match-date match-venue")
        match_venue = match_venue_el.text.strip() if match_venue_el else "N/A"

        match_date_el = soup.find(class_="match-info-date") or soup.find(
            class_="match-date"
        )
        match_date = match_date_el.text.strip() if match_date_el else "N/A"

        # 2) Teams
        teams_name = []
        teams_el = soup.find_all(class_="form-team-name")
        for team_el in teams_el:
            teams_name.append(team_el.get_text(strip=True) if team_el else "N/A")

        # 3) Series Name
        series_name_el = soup.find(class_="s-name")
        series_name = series_name_el.text.strip() if series_name_el else "N/A"

        # 4) Toss info
        toss_el = soup.find(class_="toss-wrap")
        if toss_el:
            toss_p = toss_el.find("p")
            toss_info = toss_p.get_text(strip=True) if toss_p else "N/A"
        else:
            toss_info = "N/A"

        # 5) Head-to-head
        head_to_head = []
        team1_wins_el = soup.find(class_="team1-wins")
        team2_wins_el = soup.find(class_="team2-wins")
        head_to_head.append(team1_wins_el.text if team1_wins_el else "N/A")
        head_to_head.append(team2_wins_el.text if team2_wins_el else "N/A")

        # 6) Match result or highlight
        match_result = []
        matches = soup.find_all(class_="global-match-card gmc-without-logo")
        for m in matches:
            match_result.append(m.text.strip() if m else "N/A")

        # 7) Additional weather/venue stats
        table_el = soup.find(class_="table table-borderless colHeader")
        table = table_el.text.strip() if table_el else "N/A"

        venue_details_el = soup.find(class_="align-center weather-wrap")
        venue_details = venue_details_el.text.strip() if venue_details_el else "N/A"

        venue_stats_el = soup.find(class_="venue-left-wrapper")
        venue_stats = venue_stats_el.text.strip() if venue_stats_el else "N/A"

        pace_vs_spin_on_venue_el = soup.find(class_="venue-pace-wrap")
        pace_vs_spin_on_venue = (
            pace_vs_spin_on_venue_el.text.strip() if pace_vs_spin_on_venue_el else "N/A"
        )

        # Compile
        match_info_data = {
            "match_venue": match_venue,
            "match_date": match_date,
            "teams_name": teams_name,
            "series_name": series_name,
            "toss_info": toss_info,
            "head_to_head": head_to_head,
            "match_result": match_result,
            "scorecard_table": table,
            "venue_details": venue_details,
            "venue_stats": venue_stats,
            "pace_vs_spin_on_venue": pace_vs_spin_on_venue,
        }

        return match_info_data

    finally:
        driver.quit()


# ------------------------------------------------------------------------------
# 3) SCRAPE “LIVE” TAB => /live
# ------------------------------------------------------------------------------
def scrape_live_data(live_url):
    """
    Scrapes data from the "Live" tab at (match_url + "/live").
    Useful when the match is actually in progress; might contain
    ball-by-ball updates, etc.
    """

    webdriver_path = "chromedriver.exe"
    service = Service(webdriver_path)
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(service=service, options=options)
    driver.get(live_url)  # e.g., "https://crex.live/<match-id>/live"

    try:
        # Wait for some element that indicates live data has loaded
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "live-wrapper"))
        )

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Example: Extract some live commentary or data
        # If you have a container with class .live-wrapper that holds ball-by-ball:
        live_container = soup.find(class_="live-wrapper")
        live_data = live_container.get_text(strip=True) if live_container else "N/A"

        # You might have sub-sections like .ball-details, .commentary, etc.
        # Adjust accordingly

        return {"live_data": live_data}

    finally:
        driver.quit()


# ------------------------------------------------------------------------------
# 4) SCRAPE “SCORECARD” TAB => /scorecard
# ------------------------------------------------------------------------------
def get_scorecard_data(scorecard_url):
    """
    Scrapes details from the "Scorecard" tab (match_url + "/scorecard").
    For a live or upcoming match, it might show partial or different data.
    """

    webdriver_path = "chromedriver.exe"
    service = Service(webdriver_path)
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(service=service, options=options)
    driver.get(scorecard_url)  # e.g. "https://crex.live/<match-id>/scorecard"

    try:
        # Wait for a known element that indicates the scorecard is loaded
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "scorecard-wrapper"))
        )

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 1) Venue & Date
        match_venue_el = soup.find(class_="match-date match-venue")
        match_venue = match_venue_el.text.strip() if match_venue_el else "N/A"

        match_date_el = soup.find(class_="match-info-date") or soup.find(
            class_="match-date"
        )
        match_date = match_date_el.text.strip() if match_date_el else "N/A"

        # 2) Series Name
        series_name_el = soup.find(class_="s-name")
        series_name = series_name_el.text.strip() if series_name_el else "N/A"

        # 3) Toss info (if present)
        toss_el = soup.find(class_="toss-wrap")
        if toss_el:
            toss_p = toss_el.find("p")
            toss_info = toss_p.get_text(strip=True) if toss_p else "N/A"
        else:
            toss_info = "N/A"

        # 4) Head-to-head or other data
        head_to_head = []
        team1_wins_el = soup.find(class_="team1-wins")
        team2_wins_el = soup.find(class_="team2-wins")
        head_to_head.append(team1_wins_el.text if team1_wins_el else "N/A")
        head_to_head.append(team2_wins_el.text if team2_wins_el else "N/A")

        # 5) Match results
        match_result = []
        matches = soup.find_all(class_="global-match-card gmc-without-logo")
        for m in matches:
            match_result.append(m.text.strip() if m else "N/A")

        # 6) Scorecard table
        table_el = soup.find(class_="table table-borderless colHeader")
        table = table_el.text.strip() if table_el else "N/A"

        # 7) Venue details
        venue_details_el = soup.find(class_="align-center weather-wrap")
        venue_details = venue_details_el.text.strip() if venue_details_el else "N/A"

        venue_stats_el = soup.find(class_="venue-left-wrapper")
        venue_stats = venue_stats_el.text.strip() if venue_stats_el else "N/A"

        pace_vs_spin_on_venue_el = soup.find(class_="venue-pace-wrap")
        pace_vs_spin_on_venue = (
            pace_vs_spin_on_venue_el.text.strip() if pace_vs_spin_on_venue_el else "N/A"
        )

        scorecard_data = {
            "match_venue": match_venue,
            "match_date": match_date,
            "series_name": series_name,
            "toss_info": toss_info,
            "head_to_head": head_to_head,
            "match_result": match_result,
            "scorecard_table": table,
            "venue_details": venue_details,
            "venue_stats": venue_stats,
            "pace_vs_spin_on_venue": pace_vs_spin_on_venue,
        }

        return scorecard_data

    finally:
        driver.quit()


# def scrape_squads_with_clicks(match_url):
#     webdriver_path = "chromedriver.exe"
#     service = Service(webdriver_path)
#     options = Options()
#     options.add_argument("--headless")
#     options.add_argument("--disable-gpu")
#     options.add_argument("--no-sandbox")

#     driver = webdriver.Chrome(service=service, options=options)
#     squads_url = match_url
#     driver.get(squads_url)

#     data = {}

#     try:
#         # Wait for the team buttons to show up
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_all_elements_located((By.CLASS_NAME, "playingxi-button"))
#         )

#         # Collect all team buttons
#         team_buttons = driver.find_elements(By.CLASS_NAME, "playingxi-button")

#         all_teams = []

#         for btn in team_buttons:
#             # Get the team name from the button text
#             team_name = btn.text.strip()

#             # Click the button to reveal that team's squad
#             btn.click()

#             # Wait for the playing XI to show
#             WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.CLASS_NAME, "info-right-wrapper"))
#             )

#             # Parse the updated DOM
#             page_content = driver.page_source
#             soup = BeautifulSoup(page_content, "html.parser")

#             # Now find playingxi-card / on-bench-wrap for this team
#             playing_div = soup.find("div", class_="playingxi-card")
#             bench_div = soup.find("div", class_="playingxi-card on-bench-wrap")

#             # ----- Extract Playing XI -----
#             playing_players = []
#             if playing_div:
#                 # Each player row might look like:
#                 # <div class="player-row">
#                 #     <div class="p-name">Player Name</div>
#                 #     <div class="bat-ball-type">All-rounder</div>
#                 # </div>
#                 player_rows = playing_div.find_all("div", class_="player-row")
#                 for row in player_rows:
#                     name_div = row.find("div", class_="p-name")
#                     type_div = row.find("div", class_="bat-ball-type")

#                     player_name = name_div.get_text(strip=True) if name_div else "N/A"
#                     player_type = type_div.get_text(strip=True) if type_div else "N/A"

#                     playing_players.append(
#                         {"player_name": player_name, "player_type": player_type}
#                     )

#             # ----- Extract Bench Players -----
#             bench_players = []
#             if bench_div:
#                 bench_rows = bench_div.find_all("div", class_="player-row")
#                 for row in bench_rows:
#                     name_div = row.find("div", class_="p-name")
#                     type_div = row.find("div", class_="bat-ball-type")

#                     player_name = name_div.get_text(strip=True) if name_div else "N/A"
#                     player_type = type_div.get_text(strip=True) if type_div else "N/A"

#                     bench_players.append(
#                         {"player_name": player_name, "player_type": player_type}
#                     )

#             # Store the results for each team
#             all_teams.append(
#                 {
#                     "team_name": team_name,
#                     "playing_11": playing_players,
#                     "on_bench": bench_players,
#                 }
#             )

#         data["squads"] = all_teams if all_teams else "N/A"

#     finally:
#         driver.quit()

#     return data



live_data, upcoming_data = get_match_data()
print("Live Data:", live_data)
print("Upcoming Data:", upcoming_data)

# 2) If you want to pick a match (live or upcoming) and fetch data:
# For demonstration, pick the first upcoming match
if upcoming_data:
    base_match_url = upcoming_data[0]["link"]

    # Match Info => /info
    print("\n--- Match Info ---")
    info_url = base_match_url + "/info"
    info_data = scrape_match_info(info_url)
    print(info_data)

    # Scorecard => /scorecard
    print("\n--- Scorecard ---")
    scorecard_url = base_match_url + "/scorecard"
    sc_data = get_scorecard_data(scorecard_url)
    print(sc_data)

    # Live => /live (usually only relevant if match is currently live)
    # But we can still attempt to scrape it
    print("\n--- Live Tab ---")
    live_tab_url = base_match_url + "/live"
    live_tab_data = scrape_live_data(live_tab_url)
    print(live_tab_data)

# 3) If there's a live match, do similarly:
if live_data:
    base_match_url = live_data[0]["link"]

    print("\n--- Live Match Info (/info) ---")
    info_data = scrape_match_info(base_match_url + "/info")
    print(info_data)

    print("\n--- Live Match Scorecard (/scorecard) ---")
    sc_data = get_scorecard_data(base_match_url + "/scorecard")
    print(sc_data)

    print("\n--- Live Match Live Tab (/live) ---")
    live_tab_data = scrape_live_data(base_match_url + "/live")
    print(live_tab_data)

Live Data: [{'live': True, 'name': ['BRH', 'SYS'], 'over': ['20.0', '8.4'], 'scores': ['138/9', '72/2'], 'link': 'https://crex.live/scoreboard/QCU/1MV/15th-Match/4J/4O/brh-vs-sys-15th-match-big-bash-league-2024-25/live'}]
Upcoming Data: [{'time_start': '6:45 PM', 'type': '40th T10, Emirates D10 2024-25', 'name': ['Fujairah', 'Emirates Blues'], 'link': 'https://crex.live/scoreboard/SLM/1PZ/40th-Match/DD/DH/emb-vs-fuj-40th-match-emirates-d10-league-2024-25/info'}, {'time_start': '9:00 PM', 'type': '41st T10, Emirates D10 2024-25', 'name': ['Ajman', 'Sharjah'], 'link': 'https://crex.live/scoreboard/SLN/1PZ/41st-Match/DF/DG/ajm-vs-sha-41st-match-emirates-d10-league-2024-25/info'}, {'time_start': '9:30 PM', 'type': '6th T10, Barbados T10 2024-25', 'name': ['Warriors BRB', 'Pelicans'], 'link': 'https://crex.live/scoreboard/SN3/1Q1/6th-Match/10S/10T/pel-vs-war-6th-match-barbados-t10-2024-25/info'}, {'time_start': '11:15 PM', 'type': '42nd T10, Emirates D10 2024-25', 'name': ['Emirates Blues',

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79BCAFB05+28789]
	(No symbol) [0x00007FF79BC186E0]
	(No symbol) [0x00007FF79BAB592A]
	(No symbol) [0x00007FF79BB0930E]
	(No symbol) [0x00007FF79BB095FC]
	(No symbol) [0x00007FF79BB528A7]
	(No symbol) [0x00007FF79BB2F47F]
	(No symbol) [0x00007FF79BB4F654]
	(No symbol) [0x00007FF79BB2F1E3]
	(No symbol) [0x00007FF79BAFA938]
	(No symbol) [0x00007FF79BAFBAA1]
	GetHandleVerifier [0x00007FF79BFE933D+3410093]
	GetHandleVerifier [0x00007FF79BFFE7DD+3497293]
	GetHandleVerifier [0x00007FF79BFF2A73+3448803]
	GetHandleVerifier [0x00007FF79BD77BBB+848171]
	(No symbol) [0x00007FF79BC23C3F]
	(No symbol) [0x00007FF79BC1F6E4]
	(No symbol) [0x00007FF79BC1F87D]
	(No symbol) [0x00007FF79BC0ED49]
	BaseThreadInitThunk [0x00007FF99142E8D7+23]
	RtlUserThreadStart [0x00007FF99265FBCC+44]


In [1]:
import json
import streamlit as st
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
import pytz


# ----------------------------------------------------------------------
# 1) SCRAPE MAIN FIXTURE LIST (live vs. upcoming)
# ----------------------------------------------------------------------
def get_match_data():
    """
    Scrapes the main fixture list page (https://crex.live/fixtures/match-list).
    Returns two lists:
      - live_data: Info about currently live matches
      - upcoming_data: Info about future matches
    """
    webdriver_path = "chromedriver.exe"
    service = Service(webdriver_path)
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(service=service, options=options)
    url = "https://crex.live/fixtures/match-list"
    driver.get(url)

    live_data = []
    upcoming_data = []

    try:
        # === Make Selenium WAIT up to 10 seconds for the match cards ===
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "match-card-container"))
        )

        # Now the page (or at least the match-card-container) is loaded
        soup = BeautifulSoup(driver.page_source, "html.parser")
        matches = soup.find_all(class_="match-card-container")

        for match in matches:
            # Check if match is LIVE
            if match.find(class_="liveTag"):
                link_tag = match.find("a", href=True)
                href = "https://crex.live" + link_tag["href"] if link_tag else ""

                teams_div = match.find_all("div", class_="team-info")
                team_name = []
                team_overs = []
                team_scores = []

                for t_div in teams_div:
                    name_el = t_div.find(class_="team-name")
                    over_el = t_div.find(class_="total-overs")
                    score_el = t_div.find(class_="team-score")

                    name = name_el.text.strip() if name_el else "N/A"
                    over = over_el.text.strip() if over_el else "Yet to bat"
                    score = score_el.text.strip() if score_el else "N/A"

                    team_name.append(name)
                    team_overs.append(over)
                    team_scores.append(score)

                live_data.append(
                    {
                        "live": True,
                        "name": team_name,
                        "over": team_overs,
                        "scores": team_scores,
                        "link": href,
                    }
                )

            # Otherwise, check if match is UPCOMING
            elif match.find(class_="not-started"):
                link_tag = match.find("a", href=True)
                href = "https://crex.live" + link_tag["href"] if link_tag else ""

                time_start_el = match.find(class_="start-text")
                match_type_el = match.find(class_="time")

                time_start = time_start_el.text.strip() if time_start_el else "N/A"
                match_type = match_type_el.text.strip() if match_type_el else "N/A"

                teams_div = match.find_all("div", class_="team-info")
                team_name = [
                    (
                        t_div.find(class_="team-name").text.strip()
                        if t_div.find(class_="team-name")
                        else "N/A"
                    )
                    for t_div in teams_div
                ]

                upcoming_data.append(
                    {
                        "time_start": time_start,
                        "type": match_type,
                        "name": team_name,
                        "link": href,
                    }
                )

    finally:
        driver.quit()

    return live_data, upcoming_data


# ----------------------------------------------------------------------
# 2) SCRAPE “MATCH INFO” TAB => /info
# ----------------------------------------------------------------------
def scrape_match_info(info_url):
    """
    Scrapes data from the "Match Info" tab at (match_url + "/info").
    Typically includes toss, venue, series, date, etc.
    """
    webdriver_path = "chromedriver.exe"
    service = Service(webdriver_path)
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(service=service, options=options)
    driver.get(info_url)

    try:
        # Increase timeout to allow slow loads

        EC.presence_of_element_located((By.CLASS_NAME, "match-info-card"))

        soup = BeautifulSoup(driver.page_source, "html.parser")

        match_venue_el = soup.find(class_="match-date match-venue")
        match_venue = match_venue_el.text.strip() if match_venue_el else "N/A"

        match_date_el = soup.find(class_="match-info-date") or soup.find(
            "div", class_="match-date"
        )
        match_date = match_date_el.text.strip() if match_date_el else "N/A"

        teams_name = []
        teams_el = soup.find_all(class_="form-team-name")
        for team_el in teams_el:
            teams_name.append(team_el.get_text(strip=True) if team_el else "N/A")

        series_name_el = soup.find(class_="s-name")
        series_name = series_name_el.text.strip() if series_name_el else "N/A"

        toss_el = soup.find(class_="toss-wrap")
        if toss_el:
            toss_p = toss_el.find("p")
            toss_info = toss_p.get_text(strip=True) if toss_p else "N/A"
        else:
            toss_info = "N/A"

        head_to_head = []
        team1_wins_el = soup.find(class_="team1-wins")
        team2_wins_el = soup.find(class_="team2-wins")
        head_to_head.append(team1_wins_el.text if team1_wins_el else "N/A")
        head_to_head.append(team2_wins_el.text if team2_wins_el else "N/A")

        match_result = []
        matches = soup.find_all(class_="global-match-card gmc-without-logo")
        for m in matches:
            match_result.append(m.text.strip() if m else "N/A")

        table_el = soup.find(class_="table table-borderless colHeader")
        table = table_el.text.strip() if table_el else "N/A"

        venue_details_el = soup.find(class_="align-center weather-wrap")
        venue_details = venue_details_el.text.strip() if venue_details_el else "N/A"

        venue_stats_el = soup.find(class_="venue-left-wrapper")
        venue_stats = venue_stats_el.text.strip() if venue_stats_el else "N/A"

        pace_vs_spin_on_venue_el = soup.find(class_="venue-pace-wrap")
        pace_vs_spin_on_venue = (
            pace_vs_spin_on_venue_el.text.strip() if pace_vs_spin_on_venue_el else "N/A"
        )

        match_info_data = {
            "match_venue": match_venue,
            "match_date": match_date,
            "teams_name": teams_name,
            "series_name": series_name,
            "toss_info": toss_info,
            "head_to_head": head_to_head,
            "match_result": match_result,
            "scorecard_table": table,
            "venue_details": venue_details,
            "venue_stats": venue_stats,
            "pace_vs_spin_on_venue": pace_vs_spin_on_venue,
        }
        return match_info_data
    finally:
        driver.quit()


# ----------------------------------------------------------------------
# 3) SCRAPE “LIVE” TAB => /live
# ----------------------------------------------------------------------
def scrape_live_data(live_url, match_info=None):
    """
    Scrapes data from the "Live" tab at (match_url + "/live").
    Extracts team names and scores, including runs and overs.
    If Team 2's name is missing or invalid, it falls back to match_info.
    """
    webdriver_path = "chromedriver.exe"
    service = Service(webdriver_path)
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(service=service, options=options)
    driver.get(live_url)

    try:
        # Wait for the live-score-card to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "live-score-card"))
        )

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Find the live-score-card container
        live_container = soup.find(class_="live-score-card")
        if not live_container:
            return {"live_data": "N/A"}

        # Initialize data structure
        live_data = {}

        # Extract data for both teams
        team_divs = live_container.find_all(class_="team-content")
        for index, team_div in enumerate(team_divs):
            team_name_el = team_div.find(class_="team-name")
            score_div = team_div.find(class_="team-score")

            # Extract runs and overs from spans
            if score_div:
                spans = score_div.find_all("span")
                runs = spans[0].text.strip() if len(spans) > 0 else "N/A"
                overs = spans[1].text.strip() if len(spans) > 1 else "N/A"
            else:
                runs = "N/A"
                overs = "N/A"

            # Extract team name
            team_name = team_name_el.text.strip() if team_name_el else "N/A"

            # Validate Team 2's name
            if index == 1 and (not team_name or team_name.lower() == "team"):
                # Fallback to match_info if provided
                if (
                    match_info
                    and "teams_name" in match_info
                    and len(match_info["teams_name"]) > 1
                ):
                    team_name = match_info["teams_name"][1]

            # Team data
            live_data[f"team_{index + 1}"] = {
                "name": team_name,
                "runs": runs,
                "overs": overs,
            }

        return live_data

    except TimeoutException:
        print("Timeout: live-score-card not found on live page.")
        return {"live_data": "N/A"}
    finally:
        driver.quit()


# ----------------------------------------------------------------------
# 4) SCRAPE “SCORECARD” TAB => /scorecard
# ----------------------------------------------------------------------
# ----------------------------------------------------------------------
# 4) SCRAPE “SCORECARD” TAB => /scorecard
# ----------------------------------------------------------------------
def scrape_partnerships(soup):
    """
    Extract the Partnerships section from the scorecard page.
    Args:
        soup (BeautifulSoup): The parsed HTML of the scorecard page.
    Returns:
        list: A list of dictionaries containing partnership data.
    """
    partnerships_data = []

    # Locate the partnerships section by class
    partnership_section = soup.find("div", class_="partnership-section")
    if not partnership_section:
        print("Partnerships section not found.")
        return partnerships_data

    # Parse each partnership block
    partnership_blocks = partnership_section.find_all("div", class_="p-section-wrapper")
    for block in partnership_blocks:
        # Extract wicket information
        wicket_info = block.find("div", class_="p-wckt-info")
        wicket = wicket_info.text.strip() if wicket_info else "N/A"

        # Extract partnership details
        partnership_info = block.find("div", class_="p-info-wrapper")
        if not partnership_info:
            continue

        data_points = partnership_info.find_all("div", class_="p-data")
        if len(data_points) >= 3:
            batter1 = data_points[0].find("p").text.strip()
            batter1_stats = (
                data_points[0].find("span", class_="run-highlight").text.strip()
            )

            total_runs = data_points[1].find("p", class_="p-runs").text.strip()

            batter2 = data_points[2].find("p").text.strip()
            batter2_stats = (
                data_points[2].find("span", class_="run-highlight").text.strip()
            )

            partnerships_data.append(
                {
                    "wicket": wicket,
                    "batter1": batter1,
                    "batter1_stats": batter1_stats,
                    "total_runs": total_runs,
                    "batter2": batter2,
                    "batter2_stats": batter2_stats,
                }
            )

    return partnerships_data


def get_scorecard_data(scorecard_url):
    """
    Scrapes details from the "Scorecard" tab (match_url + "/scorecard").
    Extracts batting, bowling, fall of wickets, and partnerships sections dynamically.
    """
    webdriver_path = "chromedriver.exe"
    service = Service(webdriver_path)
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(service=service, options=options)
    driver.get(scorecard_url)

    try:
        # Wait for the scorecard page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "score"))
        )

        soup = BeautifulSoup(driver.page_source, "html.parser")
        scorecard_data = {
            "batting": [],
            "bowling": [],
            "fall_of_wickets": [],
            "partnerships": [],
        }

        # Scrap batting and bowling sections
        table_headings = soup.find_all("div", class_="table-heading")
        for table_heading in table_headings:
            heading_text = table_heading.find("h3").text.strip()

            # Find the next sibling div containing the score-card
            score_card = table_heading.find_next_sibling(
                "div", class_="card score-card"
            )
            if not score_card:
                continue  # Skip if no score-card found

            score_table = score_card.find("table", class_="bowler-table")
            if not score_table:
                continue  # Skip if no table found

            rows = score_table.find("tbody").find_all("tr")
            section_data = []
            if heading_text.lower() == "batting":
                for row in rows:
                    cells = row.find_all("td")
                    if len(cells) >= 6:
                        section_data.append(
                            {
                                "batter": cells[0]
                                .find("span", class_="player-name")
                                .text.strip(),
                                "runs": cells[1].text.strip(),
                                "balls": cells[2].text.strip(),
                                "fours": cells[3].text.strip(),
                                "sixes": cells[4].text.strip(),
                                "strike_rate": cells[5].text.strip(),
                            }
                        )
                scorecard_data["batting"].append(section_data)

            elif heading_text.lower() == "bowling":
                for row in rows:
                    cells = row.find_all("td")
                    if len(cells) >= 6:
                        section_data.append(
                            {
                                "bowler": cells[0]
                                .find("span", class_="player-name")
                                .text.strip(),
                                "overs": cells[1].text.strip(),
                                "maidens": cells[2].text.strip(),
                                "runs_conceded": cells[3].text.strip(),
                                "wickets": cells[4].text.strip(),
                                "economy": cells[5].text.strip(),
                            }
                        )
                scorecard_data["bowling"].append(section_data)

        # Scrap Fall of Wickets
        scorecard_data["fall_of_wickets"] = scrape_fall_of_wickets(soup)

        # Scrap Partnerships
        scorecard_data["partnerships"] = scrape_partnerships(soup)

        return scorecard_data

    except TimeoutException:
        # print("Timeout: Unable to load scorecard.")
        return {"Match not started Yet"}
    finally:
        driver.quit()


def scrape_fall_of_wickets(soup):
    """
    Extract the Fall of Wickets section from the scorecard page.
    Args:
        soup (BeautifulSoup): The parsed HTML of the scorecard page.
    Returns:
        list: A list of dictionaries containing fall of wickets data.
    """
    fall_of_wickets_data = []

    # Locate the "Fall of Wickets" section by heading
    fall_of_wickets_heading = soup.find("h3", text="FALL OF WICKETS")
    if not fall_of_wickets_heading:
        print("Fall of Wickets section not found.")
        return fall_of_wickets_data

    # Find the parent container of the "Fall of Wickets" table
    fall_of_wickets_section = fall_of_wickets_heading.find_next(
        "div", class_="card score-card"
    )
    if not fall_of_wickets_section:
        print("Fall of Wickets table not found.")
        return fall_of_wickets_data

    # Locate the table inside the section
    fall_of_wickets_table = fall_of_wickets_section.find("table", class_="bowler-table")
    if not fall_of_wickets_table:
        print("Fall of Wickets table structure not found.")
        return fall_of_wickets_data

    # Parse the table rows
    rows = fall_of_wickets_table.find("tbody").find_all("tr")
    for row in rows:
        cells = row.find_all("td")
        if len(cells) >= 3:
            batsman = cells[0].find("span", class_="player-name")
            batsman_name = batsman.text.strip() if batsman else "N/A"
            score = cells[1].text.strip() if cells[1] else "N/A"
            overs = cells[2].text.strip() if cells[2] else "N/A"

            fall_of_wickets_data.append(
                {
                    "batsman": batsman_name,
                    "score": score,
                    "overs": overs,
                }
            )

    return fall_of_wickets_data


# ----------------------------------------------------------------------
# 5) SCRAPE “SQUADS” => /squads (with button clicks)
# ----------------------------------------------------------------------


def scrape_squads_with_clicks(match_url):
    """
    Scrapes the squads via /squads. Within .info_right_wrapper there are:
      - Buttons (class 'playingxi-button') for each team.
      - 'playingxi-card' containers for playing XI
      - 'playingxi-card on-bench-wrap' containers for bench
      - Rows with class 'playingxi-card-row', each containing .p-name and .bat-ball-type
    """

    webdriver_path = "chromedriver.exe"
    service = Service(webdriver_path)
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(service=service, options=options)
    squads_url = match_url  # E.g., https://crex.live/match-id/squads
    driver.get(squads_url)

    data = {}

    try:

        # Wait for .info-right-wrapper
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, "info-right-wrapper"))
        )

        # If even .info-right-wrapper never appears, just fallback

        # 2) Wait or check for the team buttons

        WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "playingxi-button"))
        )

        # If even .info-right-wrapper never appears, just fallback

        # 3) Grab all 'playingxi-button' elements
        try:
            team_buttons = driver.find_elements(By.CLASS_NAME, "playingxi-button")
        except TimeoutException:
            return {"squads": "N/A"}
        all_teams = []

        # 4) For each button (Team A, Team B, etc.)
        for btn in team_buttons:
            team_name = btn.text.strip()

            # 4a) Use JavaScript click to avoid potential overlay issues
            try:
                driver.execute_script("arguments[0].click();", btn)
            except ElementClickInterceptedException:
                # If still intercepted, we can attempt scroll-into-view or handle an ad
                driver.execute_script("arguments[0].scrollIntoView(true);", btn)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", btn)

            # 4b) Grab updated DOM
            page_content = driver.page_source
            soup = BeautifulSoup(page_content, "html.parser")

            # 5) Identify containers for playing XI vs bench
            playing_div = soup.find("div", class_="playingxi-card")
            bench_div = soup.find("div", class_="playingxi-card on-bench-wrap")

            playing_players = []
            if playing_div:
                # 5a) Each row is .playingxi-card-row
                rows = playing_div.find_all("div", class_="playingxi-card-row")
                for row in rows:
                    name_div = row.find("div", class_="p-name")
                    type_div = row.find("div", class_="bat-ball-type")
                    player_name = name_div.get_text(strip=True) if name_div else "N/A"
                    player_type = type_div.get_text(strip=True) if type_div else "N/A"
                    playing_players.append(
                        {"player_name": player_name, "player_type": player_type}
                    )

            bench_players = []
            if bench_div:
                # 5b) Same logic for bench rows
                rows = bench_div.find_all("div", class_="playingxi-card-row")
                for row in rows:
                    name_div = row.find("div", class_="p-name")
                    type_div = row.find("div", class_="bat-ball-type")
                    player_name = name_div.get_text(strip=True) if name_div else "N/A"
                    player_type = type_div.get_text(strip=True) if type_div else "N/A"
                    bench_players.append(
                        {"player_name": player_name, "player_type": player_type}
                    )

            # 6) Append data for this team
            all_teams.append(
                {
                    "team_name": team_name,
                    "playing_11": playing_players,
                    "on_bench": bench_players,
                }
            )

        # 7) Final result
        data["squads"] = all_teams if all_teams else "N/A"

    finally:
        driver.quit()

    return data


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF756D3FB05+28789]
	(No symbol) [0x00007FF756CA86E0]
	(No symbol) [0x00007FF756B4592A]
	(No symbol) [0x00007FF756B9930E]
	(No symbol) [0x00007FF756B995FC]
	(No symbol) [0x00007FF756BE28A7]
	(No symbol) [0x00007FF756BBF47F]
	(No symbol) [0x00007FF756BDF654]
	(No symbol) [0x00007FF756BBF1E3]
	(No symbol) [0x00007FF756B8A938]
	(No symbol) [0x00007FF756B8BAA1]
	GetHandleVerifier [0x00007FF75707933D+3410093]
	GetHandleVerifier [0x00007FF75708E7DD+3497293]
	GetHandleVerifier [0x00007FF757082A73+3448803]
	GetHandleVerifier [0x00007FF756E07BBB+848171]
	(No symbol) [0x00007FF756CB3C3F]
	(No symbol) [0x00007FF756CAF6E4]
	(No symbol) [0x00007FF756CAF87D]
	(No symbol) [0x00007FF756C9ED49]
	BaseThreadInitThunk [0x00007FF99976E8D7+23]
	RtlUserThreadStart [0x00007FF99B25FBCC+44]
